# CLeaning 99aana data

# General FAQs
There are a lot of units of measurements used in Nepalese Land Measurements in Nepal. They are 
Ropani Aana Paisa Dam Katha Dhur etc and we convert them in square feet for a general unit of
measurement. 
<br>
Ropani aaana paisa and dam ususally come together or only by themselves or in a combination.
<br>
Katha and dhur usually come together or only by themselves.

In [107]:
import pandas as pd
# pd.set_option('display.max_colwidth', None)

In [108]:
df = pd.read_csv('../csv_files/raw/99aana.csv',encoding='utf-8')

In [109]:
# remove sq m
df['area'] =  df['area'].apply(lambda x: x.split('sq m')[1])

In [110]:
pd.set_option('display.max_rows', None)
#dhur/dhir paisa kattha haat aana daam
# dealing with ropani-aana-paisa-dam first 
def clean_dashed(x):
    area = 0 # in sq feet
    if x.count('-') == 2:
        return None
    if x.count('-') == 3:
        y = x.split('-')
        # converting the splited value into integer
        for i in range(0, len(y)):
            y[i] = float(y[i])
        area = area + 5476 * y[0]   #ropani to sq ft
        area = area + 342.25 * y[1] #aana to sq ft
        area = area + 85.56 * y[2]  #paisa to sq ft
        area = area + 21.39 * y[3]  #dam to sq ft
        area = round(area,2)
        return str(area)
    else:
        return x
df['area'] =  df['area'].apply(clean_dashed)
df.dropna(subset=['area'],inplace = True)

In [111]:
# clean kattha and dhur
def clean_katha_and_dhur(x):
    area = 0 # in sq ft
    if x == None:
        return x
    x = x.lower()  # conversion to lowercase to make it convinient if a substring exists in a string.
    # Cleaned spelling mistakes with vscode
    # 1 Katha is equal to 3645 sq ft.
    # 1 Dhur is equal to 182.25 sq ft.
    if 'katha' in x and 'dhur' in x:
        katha = float(x.split('katha')[0].strip())
        dhur = float(x.split('katha')[1].split('dhur')[0].strip())
        area = area + katha * 3645
        area = area + dhur * 182.25

        area = round(area,2)
        return str(area)
        
    if 'katha' in x:
        katha = float(x.split('katha')[0])
        area = area + 3645 * katha
        area = round(area,2)
        return str(area)
    if 'dhur' in x:
        dhur = float(x.split('dhur')[0])
        area = area + dhur * 182.25
        area = round(area,2)
        return str(area)
    return x
df['area'] = df['area'].apply(clean_katha_and_dhur)

In [112]:
# clean ropani aana paisa dam
# 1 Ropani = sq feet
# 1 Aana = 342.25 sq feet
# 1 Paisa = 85.56 sq feet
# 1 Dam = sq feet
# 3 aana 3 paisa
# All data containing dam was removed due to easier data cleaning. Now only dealing with aana and paisa
def clean_RAPD(x):
    area = 0
    x = x.lower()
    if 'aana' in x and 'paisa' in x:
        aana = float(x.split('aana')[0].strip())
        paisa = float(x.split('aana')[1].split('paisa')[0].strip())
        area = area + aana * 342.25
        area = area + paisa * 85.56
        area = round(area,2)
        return str(area)
    if 'aana' in x:
        aana = float(x.split('aana')[0].strip())
        area = area + aana * 342.25
        area = round(area,2)
        return str(area)
    return x
    # paisa does not come on it's own so we don't to write code for only paisa data edge case
df['area'] = df['area'].apply(clean_RAPD)
